# Data Cleaning
---

In [1]:
import pandas as pd 
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Sales
---
Description: advertising and sales data by ASIN, by country, by day for 2022.  
Obs：18,576,508

- ASIN: VARCHAR Amazon sales identification number. Similar to a SKU or UPC.
- VENDOR_ID: INTEGER Brand identification internal to Pattern
- COUNTRY_CODE: VARCHAR Code of country of sale. E.g., US = United States. Full dictionary for codes can be provided if needed.
- ORDERING_REGION: VARCHAR Code of region of shared inventory. E.g., EU = European Union, as inventory held in Germany can be sold in France.
- DATE: DATE Date of sale OR date of click. Most sales for a click occur on the day of the click, but there can be a delay. E.g., an attributed sale, but no total sales that day.
- TOTAL_REV: FLOAT Total revenue of units sold for that ASIN, country_code, date.
- TOTAL_UNITS: INTEGER Total units sold for that ASIN, country_code, date.
- ATTR_REV: FLOAT Revenue that came as a consequence of clicking on an Ad as claimed by Amazon. Major caveats here: 1) if a customer clicks on your ad and buys ANY product from your company that unit / revenue will be counted as part of attribution. 2) Attribution continues for 7 days. 3) If a customer clicks an ad and would have purchased your product anyway (e.g., ad is shown in first slot and identical listing with same product is shown organically in second slot).
- ATTR_UNITS: INTEGER Same description as ATTR_REV but units sold.

---
Cleaning Step:
- Step 1:  
Remove products with asin='NOASIN'

- Step 2:  
Drop if missing vendor_id

- Step 3: <br>
Drop records if only advertised in Canada

In [2]:
Sales0 = pd.read_csv('ucla_study_ads_and_total_units.csv') 

In [3]:
Sales0.date.max()

'2023-01-10'

In [4]:
Sales0.date.min()

'2022-01-02'

In [5]:
# Remove rows containing NOASIN
Sales_noasin = Sales0[Sales0['asin'].str.contains(pat = 'NOASIN', regex = False) == True]
Sales1 = Sales0.drop(Sales_noasin.index)

# Vendor_id: 0 --> NaN, and remove
Sales1['vendor_id'] = Sales1['vendor_id'].replace(0,np.nan) 
Sales2 = Sales1.dropna(subset = ['vendor_id'])

## Ads
---
Description: Advertising performance and organic rank on page for both product and best rank by brand for select products and brands that are part of the Destiny AI advertising system. <br>
Date Range: 7/2/2018 - 7/1/2023 <br>
Obs: 16,262,275

- COUNTRY_CODE: VARCHAR Code of country of advertising and/or search placement. 
- BRAND_ID: INTEGER Brand identification internal to Pattern.
- KEYWORD: VARCHAR Keyword advertised on and/or search placement.
- DATE: DATE Date of click for advertising (not necessarily sale) and date of search placement.
- PARENT_ASIN: VARCHAR Roll up of various ASINs to parent product. E.g., same socks but different colors and sizes. If parent asin not exist, then the parent ASIN will repeat the ASIN.
- ASIN: VARCHAR Amazon sales identification number. Similar to a SKU or UPC.
- OVERALL_RANK_ORGANIC: Integer. Organic placement on search page by Amazon. E.g., 3 will be the third organic slot on the page. Because of the high number of ads on Amazon this could be down the page, especially if it's not in the top 4. If NULL then either not scraped or product not ranked. (See SERP_SCRAPED)
- MIN_BRAND_RANK: Integer. Best organic ranking by any product from the brand on the keywords.
- MIN_PARENT_RANK: Integer. Best organic ranking by any sibling ASIN from the parent_asin on the keywords.
- IMPRESSIONS: Integer. The number of times your ad was served somewhere on the search page in a day.
- CLICK: Integer. The number of clicks your ad received on the search page in a day.
- CONVERSIONS: Integer. The number of times one of your products was purchased within 7 days of clicking on an ad.
- SERP_SCRAPED: Boolean. Was overall_rank_organic scraped on that day?
 
---
Cleaning Step:
 - Step 1:   
   Remove duplicate records  
   
 - Step 2:   
    Limit to US products.
   
 - Step 3:   
   Limit the ads after 2022-01-02  
   
 - Step 4:   
   Remove duplicate records, only keep the records with the lowest 'overall_organic_rank' 
   
 - Step 5:  
   Check the data with different values of 'serp_scraped'  
   
 - Step 6:  
   Remove records with abnormal values in 'impressions'. 
   
Remaining Obs: 4,718,370 rows

In [6]:
Ads0 = pd.read_csv('ucla_study_selected_organic_rank_ads.csv')

In [7]:
Ads0.date.max()

'2023-01-07'

In [8]:
Ads0.date.min()

'2018-07-02'

### Step 1: Remove duplicate records
---

In [9]:
# Remove duplicate ads data
Ads_dup = Ads0.duplicated()
Ads1 = Ads0[- Ads_dup]

In [10]:
Ads1.head()

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
0,US,5,lithium orotate,2022-08-27,B009LI7VRC,B009LI7VRC,53.00,44.00,1.00,42.00,1.00,0.00,True
1,US,3,pure encapsulations vitamin d3,2022-10-15,B08NRVQ3F9,B0017QCNUQ,75.00,1.00,1.00,180.00,1.00,0.00,True
2,US,374,gaia herbs black elderberry,2023-01-06,B0BGMJ9WNY,B089LCQXZB,1.00,1.00,1.00,2.00,1.00,1.00,True
3,US,430,duralactin for cats,2022-11-18,B09VLN41TL,B09VLN41TL,NaN,1.00,5.00,32.00,1.00,1.00,True
4,US,430,duralactin for cats,2022-11-18,B09VLN41TL,B09VLN41TL,5.00,1.00,5.00,32.00,1.00,1.00,True


### Step 2: Limit to US
---

In [11]:
# Find which products have been advertised in the United States 
# and which products have been advertised in Canada.
Ads_CA = Ads1[Ads1['country_code'] == 'CA']
Ads_US = Ads1[Ads1['country_code'] == 'US']

In [12]:
Ads_CA.describe()

,brand_id,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions
count,3303264.00,16437.00,29787.00,75263.00,3303264.00,3303264.00,3303264.00
mean,27.10,23.01,9.57,12.15,420.69,3.97,1.47
std,12.99,27.70,15.42,18.98,1748.03,10.62,4.76
min,3.00,1.00,1.00,1.00,-9.00,1.00,0.00
25%,29.00,4.00,1.00,2.00,6.00,1.00,0.00
50%,29.00,13.00,3.00,5.00,28.00,1.00,0.00
75%,29.00,30.00,11.00,15.00,177.00,3.00,1.00
max,134.00,144.00,143.00,144.00,175848.00,254.00,145.00


In [13]:
# Products advertised in both markets: 13
both = set(Ads_CA['asin'].unique()).intersection(set(Ads_US['asin'].unique())) 
# Products advertised only in CA: 327
CA_only = set(Ads_CA['asin'].unique()).difference(both) 
# Products advertised only in US: 2555
US_only = set(Ads_US['asin'].unique()).difference(both)

In [14]:
# Check their brands
print(Ads1[Ads1.asin.isin(both)].brand_id.unique())
print(Ads1[Ads1.asin.isin(CA_only)].brand_id.unique())
print(Ads1[Ads1.asin.isin(US_only)].brand_id.unique())

[ 29 134]
[ 12  29 134   3]
[  5   3 374 430 356  60 518   4 103 118  58 403  11 453 426 321 437 393
  40 346 294 380 401  38  87  20   2 458 394 442 364 311 425  12 375 386
 334 353  29  30  15 302 286 131 384 440 498 436 457 348  44 481 134 406
 347 452  16 459 471 392]


In [15]:
Ads2 = Ads_US.copy()

### Step 3 Limit the time range of the ads data
---
- Because the sales data begins on January 2, 2022, we will limit the advertising data accordingly. 
- We will only keep the products that appear in the Sales dataset, as we want to be able to query the corresponding sales information for all products in Ads.

In [16]:
# Filter ads data for the US from January 2, 2022 onwards
Ads3 = Ads2[Ads2['date'] >= '2022-01-02']

In [17]:
# Select ASINs that are common to both Sales and Ads
selected = set(Ads3['asin'].unique()).intersection(set(Sales2['asin'].unique()))
print(len(selected),'products are selected.')

2368 products are selected.


In [18]:
Ads3 = Ads3[Ads3['asin'].isin(selected)]

In [19]:
Ads3.describe()

,brand_id,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions
count,5286708.00,887684.00,1577997.00,2925733.00,5286708.00,5286708.00,5286708.00
mean,141.59,24.72,13.85,7.34,272.51,2.74,0.69
std,178.31,29.76,21.18,17.01,2122.03,9.81,2.93
min,2.00,1.00,1.00,1.00,-10.00,1.00,0.00
25%,4.00,3.00,1.00,1.00,2.00,1.00,0.00
50%,12.00,13.00,5.00,1.00,10.00,1.00,0.00
75%,356.00,33.00,18.00,5.00,71.00,2.00,1.00
max,518.00,144.00,144.00,144.00,957040.00,1559.00,289.00


### Step 4 Further remove and process any remaining duplicate records
---
- In step one, we removed exact duplicate records, but we also found that some records are identical except for certain columns. Therefore, in this section, we will process these records.  
  e.g.![image.png](attachment:image.png)

In [20]:
# Check if there are any duplicate records when excluding a certain column
for col in Ads3.columns:
    if Ads3.drop(col, axis = 1).duplicated().any():
        print(col, 'Questioned')
    else:
        print(col, 'Good')

country_code Good
brand_id Good
keyword Questioned
date Questioned
parent_asin Good
asin Questioned
overall_rank_organic Questioned
min_brand_rank Good
min_parent_rank Good
impressions Good
click Good
conversions Good
serp_scraped Good


- Apart from 'asin', 'keyword', and 'date', we can find that there are duplicate records after exclusing 'overall_organic_rank'.
- As each record in the Ads dataset should be a unique value for the combination of asin-date-keyword, we need to address the issue of overall_organic_rank.

In [21]:
# Group by (asin, date, keyword) and count unique values in each group
Ads_group = Ads3.groupby(['asin', 'date', 'keyword'])
Ads_group.nunique().describe()

,country_code,brand_id,parent_asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
count,4718948.00,4718948.00,4718948.00,4718948.00,4718948.00,4718948.00,4718948.00,4718948.00,4718948.00,4718948.00
mean,1.00,1.00,1.00,0.19,0.21,0.50,1.00,1.00,1.00,1.00
std,0.00,0.00,0.00,0.39,0.41,0.50,0.00,0.00,0.00,0.00
min,1.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00
25%,1.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00
50%,1.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00
75%,1.00,1.00,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00
max,1.00,1.00,1.00,3.00,1.00,1.00,1.00,1.00,1.00,1.00


- After grouping by asin-date-keyword, for variables without missing values, the value of 1 is the normal case. For variables with missing values, if the only record within a group is missing, the value is 0. If the only record within a group is not missing, the value should be 1.
- Since we already know that there are no duplicated values when excluding min_brand_rank or min_parent_rank, these two variables do not need to be checked.
- For overall_organic_rank, values range from 0 to 3, and we can group them to investigate.

#### e.g. count == 4

In [22]:
# This table displays how many times each group of asin-date-keyword appears
# In normal circumstances, it should only appear once
count = Ads_group['brand_id'].count().reset_index().rename(columns={'brand_id': 'count'})

In [23]:
count

,asin,date,keyword,count
0,B0000225HZ,2022-03-09,batteries for nail gun,1
1,B0000225HZ,2022-03-09,paslode,1
2,B0000225HZ,2022-03-09,paslode battery,1
3,B0000225HZ,2022-03-09,passload charger and battery,1
4,B0000225HZ,2022-03-09,passload nail gun battery,1
...,...,...,...,...
4718943,B0BM4YSSNY,2023-01-03,smoker,1
4718944,B0BM4YSSNY,2023-01-05,smokers,1
4718945,B0BM4YSSNY,2023-01-06,smokers,1
4718946,B0BM4YSSNY,2023-01-07,smokers,1


In [24]:
# Select cases which appear four times
d_4 = pd.merge(Ads3, count[count['count'] == 4], 
               on = ['asin', 'date', 'keyword'], how = 'inner')
d_4

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped,count
0,US,12,genestra brands,2022-10-18,B00CFTYROE,B00CFTYROE,98.00,1.00,38.00,8.00,1.00,0.00,True,4
1,US,12,genestra brands,2022-10-18,B00CFTYROE,B00CFTYROE,84.00,1.00,38.00,8.00,1.00,0.00,True,4
2,US,12,genestra brands,2022-10-18,B00CFTYROE,B00CFTYROE,NaN,1.00,38.00,8.00,1.00,0.00,True,4
3,US,12,genestra brands,2022-10-18,B00CFTYROE,B00CFTYROE,38.00,1.00,38.00,8.00,1.00,0.00,True,4


#### e.g. count == 3

In [25]:
# Select cases which appear three times
d_3 = pd.merge(Ads3, count[count['count'] == 3], 
               on = ['asin', 'date', 'keyword'], how = 'inner')
d_3.fillna(150, inplace = True) # Fill missing values with 150 as the maximum organic_rank is 144
d_3

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped,count
0,US,374,hormone balance,2022-10-26,B00HLJO112,B0036THM7Q,150.00,4.00,1.00,4.00,1.00,0.00,True,3
1,US,374,hormone balance,2022-10-26,B00HLJO112,B0036THM7Q,5.00,4.00,1.00,4.00,1.00,0.00,True,3
2,US,374,hormone balance,2022-10-26,B00HLJO112,B0036THM7Q,4.00,4.00,1.00,4.00,1.00,0.00,True,3
3,US,3,b2 vitamins,2022-12-18,B08NRDK5FZ,B00JYFN6DU,104.00,45.00,1.00,12.00,1.00,1.00,True,3
4,US,3,b2 vitamins,2022-12-18,B08NRDK5FZ,B00JYFN6DU,95.00,45.00,1.00,12.00,1.00,1.00,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,US,3,fish oils omega 3,2022-11-14,B08NRQ4J96,B00U1V1YRQ,96.00,71.00,1.00,2.00,2.00,3.00,True,3
2660,US,3,fish oils omega 3,2022-11-14,B08NRQ4J96,B00U1V1YRQ,104.00,71.00,1.00,2.00,2.00,3.00,True,3
2661,US,3,organic zinc,2022-07-22,B08NRK2CCL,B004EMGRMK,97.00,25.00,2.00,54.00,1.00,0.00,True,3
2662,US,3,organic zinc,2022-07-22,B08NRK2CCL,B004EMGRMK,150.00,25.00,2.00,54.00,1.00,0.00,True,3


In [26]:
len(d_3[d_3['overall_rank_organic'] == 150])

888

 - There are 888(<=2664/3) groups of data.  
 - And it can be observed that all the data except for overall_rank_organic is the same. Due to filling the missing values with 150 and the number of records with a value of 150 being 888, we know that the values for each set are A, B, and NaN:  
 
   asin date keyword ... overall_organic_rank  
   xxxx xxxx xxxxxxx ... A  
   xxxx xxxx xxxxxxx ... B  
   xxxx xxxx xxxxxxx ... NaN  

#### e.g. count == 2
---
There are two cases for overall_organic_rank:
1. each group has two values: A and B
2. each group has one value A and one missing value NaN

In [27]:
# Select cases which appear twice times
d_2 = pd.merge(Ads3, count[count['count'] == 2], 
               on = ['asin', 'date', 'keyword'], how = 'inner')
d_2

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped,count
0,US,374,gaia herbs black elderberry,2023-01-06,B0BGMJ9WNY,B089LCQXZB,1.00,1.00,1.00,2.00,1.00,1.00,True,2
1,US,374,gaia herbs black elderberry,2023-01-06,B0BGMJ9WNY,B089LCQXZB,NaN,1.00,1.00,2.00,1.00,1.00,True,2
2,US,430,duralactin for cats,2022-11-18,B09VLN41TL,B09VLN41TL,NaN,1.00,5.00,32.00,1.00,1.00,True,2
3,US,430,duralactin for cats,2022-11-18,B09VLN41TL,B09VLN41TL,5.00,1.00,5.00,32.00,1.00,1.00,True,2
4,US,374,milk thistle organic,2022-07-10,B0BGT3DR2Q,B005ACNNLG,10.00,10.00,1.00,131.00,2.00,0.00,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131957,US,3,riboflavin 400mg,2022-09-25,B08NRDK5FZ,B00JYFN6DU,NaN,24.00,1.00,58.00,2.00,1.00,True,2
1131958,US,3,pure encapsulation b complex,2022-08-25,B08NRDK5FZ,B00JYFN6DU,NaN,1.00,1.00,121.00,25.00,10.00,True,2
1131959,US,3,pure encapsulation b complex,2022-08-25,B08NRDK5FZ,B00JYFN6DU,1.00,1.00,1.00,121.00,25.00,10.00,True,2
1131960,US,3,mens multivitamin,2022-11-24,B08NRJ4T3Y,B00CBYG1L0,15.00,15.00,1.00,197.00,2.00,0.00,True,2


In [28]:
d_2.groupby(['asin', 'date', 'keyword']).nunique().describe()

,country_code,brand_id,parent_asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped,count
count,565981.00,565981.00,565981.00,565981.00,565981.00,565981.00,565981.00,565981.00,565981.00,565981.00,565981.00
mean,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
std,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00
min,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
25%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
50%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
75%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
max,1.00,1.00,1.00,2.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [29]:
# A+B examples
d2_group1 = d_2.groupby(['asin', 'date', 'keyword']).nunique()
d2_group1 = d2_group1[d2_group1['overall_rank_organic'] == 2]
pd.merge(Ads3, d2_group1.reset_index()[['asin', 'date', 'keyword']], on = ['asin', 'date', 'keyword'], how = 'inner').head(6)

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
0,US,364,electric travel toothbrushes,2022-11-17,B09FS1LDK1,B07VVRYNP2,51.00,3.00,8.00,16.00,1.00,0.00,True
1,US,364,electric travel toothbrushes,2022-11-17,B09FS1LDK1,B07VVRYNP2,48.00,3.00,8.00,16.00,1.00,0.00,True
2,US,5,nmn supplement,2022-09-23,B097Z5387H,B097Z5387H,88.00,88.00,1.00,1788.00,8.00,2.00,True
3,US,5,nmn supplement,2022-09-23,B097Z5387H,B097Z5387H,98.00,88.00,1.00,1788.00,8.00,2.00,True
4,US,403,pink himalayan salt,2022-11-22,B08MTK48T4,B08MTK48T4,94.00,94.00,1.00,5.00,1.00,0.00,True
5,US,403,pink himalayan salt,2022-11-22,B08MTK48T4,B08MTK48T4,98.00,94.00,1.00,5.00,1.00,0.00,True


In [30]:
# A+NaN examples
d2_group2 = d_2.fillna(150, inplace = False).groupby(['asin', 'date', 'keyword']).nunique()
d2_group2 = d2_group2[d2_group2['overall_rank_organic'] == 2]
pd.merge(Ads3, d2_group2.reset_index()[['asin', 'date', 'keyword']], on = ['asin', 'date', 'keyword'], how = 'inner').head(6)

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
0,US,374,gaia herbs black elderberry,2023-01-06,B0BGMJ9WNY,B089LCQXZB,1.00,1.00,1.00,2.00,1.00,1.00,True
1,US,374,gaia herbs black elderberry,2023-01-06,B0BGMJ9WNY,B089LCQXZB,NaN,1.00,1.00,2.00,1.00,1.00,True
2,US,430,duralactin for cats,2022-11-18,B09VLN41TL,B09VLN41TL,NaN,1.00,5.00,32.00,1.00,1.00,True
3,US,430,duralactin for cats,2022-11-18,B09VLN41TL,B09VLN41TL,5.00,1.00,5.00,32.00,1.00,1.00,True
4,US,374,milk thistle organic,2022-07-10,B0BGT3DR2Q,B005ACNNLG,10.00,10.00,1.00,131.00,2.00,0.00,True
5,US,374,milk thistle organic,2022-07-10,B0BGT3DR2Q,B005ACNNLG,NaN,10.00,1.00,131.00,2.00,0.00,True


 - There are 565,981(<=1131962/2) groups of data.
 - And it can be observed that all the data except for overall_rank_organic is the same. 
 - Some groups have organic_rank as A,B and some combinations have organic_rank as A,NaN.

---

 - According to the definition, min_parent_rank is the minimum organic_rank for that keyword under that parent_asin, and min_brand_rank is the minimum organic_rank for that keyword under that brand_id.  
   Therefore, we can check the data and first determine whether:
   
$\textbf{overall_rank_organic} \geq \textbf{min_parent_rank} \geq \textbf{min_brand_rank}$  

- If this is true, I believe this is because the same product has been searched for multiple times, so we only keep the highest ranking instance.


 - For later verification, we perform the calculation in advance:   
   the data we finally obtain should be the sum of the number of data with count==1, half of the number of data with count==2, one-third of the number of data with count==3, and one-fourth of the number of data with count==4.   
      Therefore, the total number of such data would be 4,718,948 (4152078 + 1 + 888 + 565981).

In [31]:
# def check_ranking(df):
    # return df['overall_rank_organic'] >= df['min_parent_rank'] >= df['min_brand_rank']
def check_ranking2(df):
    return (df['overall_rank_organic'] >= df['min_parent_rank']) and (df['overall_rank_organic']>= df['min_brand_rank'])

print('d_4:', d_4.fillna(150, inplace = False).apply(check_ranking2, axis = 1).all())
print('d_3:', d_3.fillna(150, inplace = False).apply(check_ranking2, axis = 1).all())
print('d_2:', d_2.fillna(150, inplace = False).apply(check_ranking2, axis = 1).all())

d_4: True
d_3: True
d_2: True


In [32]:
Ads_copy = Ads3.copy()
Ads_copy['overall_rank_organic'].fillna(150, inplace = True)
# Keep the record with the smallest overall_rank_organic in each group
idx = Ads_copy.groupby(['asin', 'date', 'keyword'])['overall_rank_organic'].idxmin()
Ads4 = Ads3.loc[idx]

In [33]:
len(Ads4)

4718948

### Step 5 Check the data where serp_scraped is False
---
 - I noticed that there are no cases where serp_scraped is False within groups where the count is greater than 1. 
 - However, since the ranking is only for the product ranking under that certain keyword, I think the ranking is not particularly informative for DID analysis (because sales data does not distinguish between keywords).
 - In this step, we did not make any changes to the data.

In [34]:
# Ads4 is the dataset with duplicate values removed, 
# and the number of False values remains the same as before
Ads4['serp_scraped'].value_counts()

False    2731630
True     1987318
Name: serp_scraped, dtype: int64

In [35]:
Ads3['serp_scraped'].value_counts()

False    2731630
True     2555078
Name: serp_scraped, dtype: int64

In [36]:
# The company introduced robots to perform serp_scraped after July
Ads4[Ads4.serp_scraped == True].sort_values(by = 'date')

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
10584686,US,58,paslode,2022-07-01,B0000225HZ,B0000225HZ,8.00,1.00,1.00,243.00,2.00,0.00,True
4598506,US,29,feetures socks women,2022-07-01,B08CF46K47,B07QR6C4ZF,29.00,1.00,1.00,75.00,6.00,2.00,True
8865818,US,29,mini crew running socks,2022-07-01,B08CF46K47,B07QR6C4ZF,NaN,NaN,1.00,170.00,1.00,0.00,True
5214369,US,29,running socks for men,2022-07-01,B08CF46K47,B07QR6C4ZF,NaN,NaN,1.00,72.00,2.00,0.00,True
4066132,US,29,running socks for women no blisters,2022-07-01,B08CF46K47,B07QR6C4ZF,NaN,NaN,1.00,588.00,2.00,0.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523079,US,364,electric toothbrush for travel for adults,2023-01-07,B09FS7C85S,B07K379Y9V,9.00,6.00,1.00,1.00,1.00,0.00,True
5620725,US,364,electric toothbrush quip,2023-01-07,B09FS7C85S,B07K379Y9V,NaN,1.00,1.00,141.00,6.00,3.00,True
12676551,US,364,electric toothbrushes,2023-01-07,B09FS7C85S,B07K379Y9V,57.00,57.00,1.00,13.00,3.00,0.00,True
9743216,US,58,metabo framing nails,2023-01-07,B00023SEJE,B00023SEJE,NaN,NaN,1.00,24.00,1.00,0.00,True


In [37]:
Ads5 = Ads4.copy()

### Step 6 Remove abnormal records
---
 - According to the definition, impressions cannot be less than or equal to 0.
 - Since the 'conversions' of these data are very small and most of them are zero, they have little impact on sales. Therefore, I prefer to simply delete them.

In [38]:
impressions_abnormal = Ads5[Ads5['impressions'] <= 0]

In [39]:
impressions_abnormal.head()

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
8372346,US,356,big cd case,2022-11-07,B0000A1WGL,B0000A1WGL,NaN,NaN,14.00,0.00,1.00,0.00,True
14228113,US,458,hibachi grill plate,2022-12-01,B0000AQO11,B0000AQO11,NaN,NaN,47.00,0.00,1.00,0.00,False
13990206,US,294,b-complex vitamins,2022-04-23,B08MLGQ36Q,B00014D1XS,NaN,NaN,NaN,0.00,1.00,0.00,False
3698522,US,294,whole food vitamins,2022-03-28,B0B1VVJ1RD,B00014G874,NaN,NaN,NaN,0.00,1.00,0.00,False
1037472,US,294,organic vitamins men,2022-09-11,B0B1VVJ1RD,B00014G874,41.00,41.00,11.00,0.00,1.00,0.00,True


In [40]:
impressions_abnormal.describe()

,brand_id,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions
count,578.00,82.00,99.00,266.00,578.00,578.00,578.00
mean,123.29,28.71,19.73,7.43,-0.04,1.01,0.20
std,166.93,28.87,23.49,16.82,0.46,0.12,0.42
min,2.00,1.00,1.00,1.00,-10.00,1.00,0.00
25%,3.25,11.00,6.00,1.00,0.00,1.00,0.00
50%,11.00,19.00,12.00,1.00,0.00,1.00,0.00
75%,294.00,36.75,25.50,5.00,0.00,1.00,0.00
max,481.00,143.00,137.00,114.00,0.00,3.00,3.00


In [41]:
# Remove ads data with abnormal impressions values
Ads6 = Ads5[Ads5['impressions'] > 0]

In [42]:
Ads6

,country_code,brand_id,keyword,date,parent_asin,asin,overall_rank_organic,min_brand_rank,min_parent_rank,impressions,click,conversions,serp_scraped
7728826,US,58,batteries for nail gun,2022-03-09,B0000225HZ,B0000225HZ,NaN,NaN,NaN,1.00,1.00,0.00,False
4198825,US,58,paslode,2022-03-09,B0000225HZ,B0000225HZ,NaN,NaN,NaN,1.00,1.00,0.00,False
14951092,US,58,paslode battery,2022-03-09,B0000225HZ,B0000225HZ,NaN,NaN,NaN,85.00,2.00,0.00,False
9450656,US,58,passload charger and battery,2022-03-09,B0000225HZ,B0000225HZ,NaN,NaN,NaN,4.00,1.00,0.00,False
5182035,US,58,passload nail gun battery,2022-03-09,B0000225HZ,B0000225HZ,NaN,NaN,NaN,1.00,1.00,0.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750607,US,458,smoker,2023-01-03,B0BM4YSSNY,B0BM4YSSNY,NaN,5.00,NaN,9.00,1.00,0.00,True
14606379,US,458,smokers,2023-01-05,B0BM4YSSNY,B0BM4YSSNY,NaN,NaN,NaN,164.00,1.00,0.00,True
1405910,US,458,smokers,2023-01-06,B0BM4YSSNY,B0BM4YSSNY,NaN,NaN,NaN,88.00,1.00,0.00,True
9736051,US,458,smokers,2023-01-07,B0BM4YSSNY,B0BM4YSSNY,NaN,NaN,NaN,137.00,2.00,0.00,True


## Summary
---
- For the sales data, I excluded the products that were only sold in Canada, so the remaining products can be classified into three categories: 
1. those without advertising data (which we can assume have not been advertised),
2. those with advertising data and only sold in the US, and those with advertising data and 
3. sold in both the US and Canada (but since we have confirmed earlier that the conversion rate of these products in the Canadian market is low, their impact can be ignored).

In [43]:
'''Sales = Sales2[~Sales2['asin'].isin(CA_only)]
Sales.to_csv('Sales.csv', index = False)'''

"Sales = Sales2[~Sales2['asin'].isin(CA_only)]\nSales.to_csv('Sales.csv', index = False)"

In [44]:
'''Ads = Ads6.copy()
Ads.to_csv('Ads.csv', index = False)'''

"Ads = Ads6.copy()\nAds.to_csv('Ads.csv', index = False)"